In [1]:
import numpy as np 


# 辅助-根据上下界生成均匀分布的矩阵->[31,1]
def solution(lb, ub):  # lb=(31,1)
    x = []
    for l, u in zip(lb, ub):
        xi = np.random.uniform(l, u)
        x.append(xi)
    x = np.array(x).reshape(len(x), 1)
    s = np.sum(x)
    if s != 0:
        x = x / s
    else:
        # print("Repair Error: all components are 0!")
        x = solution(lb, ub) 
    return x

# 修正子代函数
def repair(y, lb, ub):  # 刚传进来的y.shape=(31, 1), max=1348.03   min=-0.007
    for i in range(len(y)):
        y[i] = max(y[i], lb[i])  #负数变为0, 
    s = np.sum(y)   
    if s != 0:
        y = y / s    #缩放使所有变量总和=1,  归一化操作
    else:
        # print("Repair Error: all components are 0!")
        y = solution(lb, ub)  #随机生成y  根据上下界生成均匀分布的解(31,1)
    return y



In [2]:
# 初始化种群p->[100, 225]   lb, ub = np.zeros((31, 1)), np.ones((31, 1))：这两行代码初始化了lb和ub，它们分别表示问题中每个资产的下界（lower bound）和上界（upper bound）。
def population(lb, ub, N):  # 种群大小N=100
    P = []
    for _ in range(N):
        P.append(solution(lb, ub))
    return P   #(100, 31)初始化种群

n = 31
lb, ub = np.zeros((n, 1)), np.ones((n, 1))  # 设置上下界->[31, 1]   n行1列全0，n行1列全1
P = population(lb, ub, 100)

In [7]:
print(P[1].shape)
print(type(P[1]))

(31, 1)
<class 'numpy.ndarray'>


In [10]:
import numpy as np


def diff(P, i, b, lb, ub, par):
    mutation_rate, mutation_strength = par[0], par[1]
    np.random.shuffle(b)
    p1 = P[i]   # （31,1)
    p2 = P[b[0]]  # 从邻居中随机选择一个个体p2   (31, 1)
    # 单点交叉操作
    crossover_point = np.random.randint(1, 31)  # 随机选择交叉点
    y = np.vstack((a[:crossover_point], b[crossover_point:]))
    y = repair(y, lb, ub)# 对变异后的个体 `y` 进行修复操作，确保它在上下界 `lb` 和 `ub` 内。

    # 变异操作（添加随机噪声）
    mutation_mask = (np.random.rand(31, 1) < mutation_rate).astype(float)
    mutation = mutation_strength * (2 * np.random.rand(31, 1) - 1)
    y += mutation * mutation_mask
    y = repair(y, lb, ub)# 对变异后的个体 `y` 进行修复操作，确保它在上下界 `lb` 和 `ub` 内。


In [2]:
import numpy as np
# 定义形状为 (31, 1) 的数组 a 和 b
a = np.random.rand(31, 1)
b = np.random.rand(31, 1)
print(len(a))
# 单点交叉操作
crossover_point = np.random.randint(1, 31)  # 随机选择交叉点
y = np.vstack((a[:crossover_point], b[crossover_point:]))


# 变异操作（添加随机噪声）
mutation_rate = 0.1  # 控制变异率
mutation_strength = 0.1  # 控制变异强度
mutation_mask = (np.random.rand(31, 1) < mutation_rate).astype(float)
mutation = mutation_strength * (2 * np.random.rand(31, 1) - 1)
y += mutation * mutation_mask

# y 现在包含了交叉和变异后的新个体
# print(y)


31


In [1]:
import numpy as np

# 定义形状为 (31, 1) 的数组 a 和 b
a = np.random.rand(31, 1)
b = np.random.rand(31, 1)

# 多点交叉操作
num_crossover_points = np.random.randint(1, 16)  # 随机选择多个交叉点，例如 1 到 15 个
crossover_points = np.sort(np.random.choice(31, num_crossover_points, replace=False))

crossover_points = np.concatenate(([0], crossover_points, [31]))  # 添加起始和结束点

y = np.empty_like(a)

for i in range(len(crossover_points) - 1):
    if i % 2 == 0:
        y[crossover_points[i]:crossover_points[i+1]] = a[crossover_points[i]:crossover_points[i+1]]
    else:
        y[crossover_points[i]:crossover_points[i+1]] = b[crossover_points[i]:crossover_points[i+1]]

# 变异操作（添加随机噪声）
mutation_rate = 0.1  # 控制变异率
mutation_strength = 0.1  # 控制变异强度
mutation_mask = (np.random.rand(31, 1) < mutation_rate).astype(float)
mutation = mutation_strength * (2 * np.random.rand(31, 1) - 1)
y += mutation * mutation_mask

# y 现在包含了多点交叉和变异后的新个体
print(y.shape)


(31, 1)


In [ ]:


# adj_lvxm-多项式变异算子(5个参数)
def adj_lvxm(P, i, b, lb, ub, par):  # i: 当前要变异的个体的索引
    alpha, beta, pm, etam, epsilon = par[0], par[1], par[2], par[3], par[4]
    np.random.shuffle(b)
    p1 = P[i]   # （31,1)
    p2 = P[b[0]]  # 从邻居中随机选择一个个体p2   (31, 1)
    y = p1 * epsilon + alpha * levy(beta, len(p1)) * (p1 - p2) * (1 - epsilon)  # 公式8    y=（31， 1）
    y = repair(y, lb, ub)# 对变异后的个体 `y` 进行修复操作，确保它在上下界 `lb` 和 `ub` 内。
    y = poly_mutation(y, lb, ub, etam, pm)  # 对变异后的个体 `y` 进行多项式变异操作，但这次使用参数 `etam` 和 `pm` 来控制多项式变异
    y = repair(y, lb, ub)  # 再次对变异后的个体 `y` 进行修复操作，确保它在上下界 `lb` 和 `ub` 内
    return y   # （31， 1)


In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class Conv1DModel(nn.Module):
    def __init__(self, input_channel=1, hidden_channel=64, output_channel=1, time_embedding_dim=31):
        super(Conv1DModel, self).__init__()
        
        # 定义时间嵌入层
        # self.time_embedding = nn.Embedding(num_embeddings=10, embedding_dim=time_embedding_dim)
        
        # 1D卷积层
        self.conv1d = nn.Conv1d(in_channels=input_channel, out_channels=hidden_channel, kernel_size=5, padding=2)
        
        # 激活函数
        self.relu = nn.ReLU()
        
        self.mid = nn.Conv1d(in_channels=hidden_channel, out_channels=hidden_channel, kernel_size=5, padding=2)
        
        # 输出层
        self.output_layer = nn.Conv1d(in_channels=hidden_channel, out_channels=output_channel, kernel_size=5, padding=2)

    def forward(self, x, t):
        # 输入张量 x 的形状应为 (batch_size, input_dim, 1)
        batch, _, len = x.shape

        # 1D卷积操作
        x = self.conv1d(x)
        
        # 嵌入时间步
        # t_embedding = self.time_embedding(t)   # (64, 31)
        # t_embedding = t_embedding.view(batch, 1, -1)


        # print("t_embedding", t_embedding.shape)
        # print("x.shape:", x.shape)

        
        # 将时间嵌入与输入 x 连接
        # x = torch.cat([x, t_embedding], dim=1)
        # x += t_embedding
        x = self.relu(x)

        x = self.mid(x)
        # x+= t_embedding
        x = self.relu(x)
        
        # 输出层
        output = self.output_layer(x)

        output = output.view(batch, -1)
        output = F.softmax(output, dim=1)  # 使用softmax确保输出的每个维度的和为1
        
        return output


# 根据instance来确定channel的大小


In [18]:
class MLPDiffusion(nn.Module):    
    def __init__(self, instance):
        # instance：数据集的id，根据数据集的不同来设置不同的hidden_channel
        super(MLPDiffusion, self).__init__()
        
        dim = [24, 32, 32, 48, 64]
        input_channel = 1
        hidden_channel = dim[instance-1] 
        output_channel = 1
        base_channels = 16
        time_emb_scale = 1.0
        time_emb_dim = base_channels
        dropout_prob = 0.5

        # 1D卷积层
        self.conv1d = nn.Conv1d(in_channels=input_channel, out_channels=hidden_channel, kernel_size=5, padding=2)
        # 激活函数
        self.relu = nn.ReLU()
        # Dropout层
        self.dropout = nn.Dropout(p=dropout_prob)

        self.mid1 = nn.Conv1d(in_channels=hidden_channel, out_channels=hidden_channel, kernel_size=5, padding=2)
        self.mid2 = nn.Conv1d(in_channels=hidden_channel, out_channels=hidden_channel, kernel_size=5, padding=2)
        # 输出层
        self.output_layer = nn.Conv1d(in_channels=hidden_channel, out_channels=output_channel, kernel_size=5, padding=2)

        # self.time_mlp = nn.Sequential(
        #     PositionalEmbedding(base_channels, time_emb_scale),
        #     # nn.Linear(base_channels, time_emb_dim),
        #     nn.SiLU(),
        #     nn.Linear(base_channels, hidden_channel),
        # )

    def forward(self, x, t):
        # 输入张量 x 的形状应为 (batch_size, input_channel, dim)
        # t的形状应该为(batch_size,)
        batch, _, len = x.shape
        # t_emb = self.time_mlp(t)  # (batch_size, outputchannel)

        #
        #  1D卷积操作
        # x = self.relu(self.conv1d(x)) + t_emb[:, :, None]
        # x = self.relu(self.mid(x)) + t_emb[:, :, None]
        # x = self.relu(self.conv1d(x)) 
        # x = self.relu(self.mid1(x)) 
        # x = self.relu(self.mid2(x))

        # 1D卷积操作
        x = self.conv1d(x)
        x = self.relu(x)
        # Dropout
        x = self.dropout(x)
        # 中间层1
        x = self.mid1(x)
        x = self.relu(x)
        x = self.dropout(x)
        # 中间层2
        x = self.mid2(x)
        x = self.relu(x)
        x = self.dropout(x)
        # 输出层

        # 输出层
        output = self.output_layer(x) 

        output = output.view(batch, -1)
        output = F.softmax(output/3.0, dim=1)  # 使用softmax确保输出的每个维度的和为1
        
        return output

In [20]:
# 测试张量
batch_size = 64
dim = 31
input_data = torch.rand(batch_size, 1, dim)  # (batch, channel, length)

t = torch.randint(low=1, high=10, size=(64,))

print(t.shape)
# 创建模型实例
# model = Conv1DModel(1) # 21249个参数
# 31 85 89 98 225
model = MLPDiffusion(1)  # 1473

# 统计模型的可训练参数数量
total_params = sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f"总共有 {total_params} 个可训练参数.")


torch.Size([64])
总共有 6073 个可训练参数.


In [14]:
# 前向传播
output = model(input_data, t)

# 输出张量的形状
print(output.shape)  # 应为 (64, 31)
print(output[0])

torch.Size([64, 31])
tensor([0.0372, 0.0369, 0.0341, 0.0334, 0.0304, 0.0322, 0.0314, 0.0315, 0.0307,
        0.0313, 0.0318, 0.0326, 0.0326, 0.0319, 0.0329, 0.0317, 0.0314, 0.0309,
        0.0317, 0.0322, 0.0304, 0.0307, 0.0316, 0.0336, 0.0311, 0.0334, 0.0326,
        0.0340, 0.0313, 0.0312, 0.0312], grad_fn=<SelectBackward0>)


In [1]:
import math
import torch
import torch.nn as nn

class PositionalEmbedding(nn.Module):
    __doc__ = r"""Computes a positional embedding of timesteps.

    Input:
        x: tensor of shape (N)
    Output:
        tensor of shape (N, dim)
    Args:
        dim (int): embedding dimension
        scale (float): linear scale to be applied to timesteps. Default: 1.0
    """

    def __init__(self, dim=128, scale=1.0):
        super().__init__()
        assert dim % 2 == 0
        self.dim = dim
        self.scale = scale

    def forward(self, x):  # torch.size([128])
        device = x.device
        half_dim = self.dim // 2
        emb = math.log(10000) / half_dim
        emb = torch.exp(torch.arange(half_dim, device=device) * -emb)
        emb = torch.outer(x * self.scale, emb)
        emb = torch.cat((emb.sin(), emb.cos()), dim=-1)
        return emb

# 参数设置
base_channels = 128
time_emb_dim = 128
time_emb_scale = 1


time_mlp = nn.Sequential(
            PositionalEmbedding(base_channels, time_emb_scale),  # (batch,) -> (batch, base_channels)
            nn.Linear(base_channels, time_emb_dim),              # (batch, base_channels) -> (batch, time_emb_dim)
            nn.SiLU(),
            nn.Linear(time_emb_dim, time_emb_dim),               # (batch, time_emb_dim) -> (batch, time_emb_dim)
        )


In [5]:
t = torch.randint(1, 30, size=(64,))

t_emb = time_mlp(t)
print(t_emb.shape)  # (batch, dim)= (64, 128)


torch.Size([64, 128])


In [ ]:

input_data = torch.rand(batch_size, 1, dim)  # (batch, channel, length)

# 参数
input_channel = 1
hidden_channel = 64
conv1d = nn.Conv1d(in_channels=input_channel, out_channels=hidden_channel, kernel_size=5, padding=2)
# 激活函数
relu = nn.ReLU()

x = relu(cov1d(input_data))
print(x.shape)


In [ ]:
# 把时间t加入到x中去
    def forward(self, x, time_emb=None, y=None):
        out = self.activation(self.norm_1(x))  # [128, 128, 32, 32]
        out = self.conv_1(out)# [128, 128, 32, 32]

        if self.time_bias is not None: 
            if time_emb is None:      # time_emb = [128, 512]
                raise ValueError("time conditioning was specified but time_emb is not passed")
            out += self.time_bias(self.activation(time_emb))[:, :, None, None]
            # time_bias : (128, 512)  -> (128, 128)


# 1. time先经过time_mlp 变为  (batch , time_emb_dim)
# 2. 对x进行conv activation处理，变为(batch, channel, len)
# 3. 再经过activation和time_bias变为 (batch, channel)
# 4. x和time_bias相加

# (batch, channel, len)    t: (batch ,channel)

In [8]:
import torch 

t_p = torch.randint(0, 3, size=(10,))
print(t_p)

t1 = torch.full([10], 3)
print(t1.shape)

noises = torch.randn(100, 31)
print(noises.shape[0])

tensor([1, 0, 0, 0, 0, 2, 2, 0, 1, 1])
torch.Size([10])
100


In [30]:
import numpy as np 
num_crossover_points = np.random.randint(1, 31)
print(num_crossover_points) 

11


In [14]:
import numpy as np
dim=31
num_crossover_points = np.random.randint(1, dim)  # 随机选择多个交叉点，例如 1 到 15 个
crossover_points = np.sort(np.random.choice(dim, num_crossover_points, replace=False))
print(num_crossover_points) # 8
print(crossover_points)  # [ 0  7  8 13 14 22 27 29]

# 在添加 0 之前检查是否已存在 0
if crossover_points[0] != 0:
    crossover_points = np.concatenate(([0], crossover_points))
if crossover_points[-1] != dim:
    crossover_points = np.concatenate([crossover_points, [dim]])

print(crossover_points) # [ 0  0  7  8 13 14 22 27 29 31]

11
[ 2  7  9 16 19 21 22 24 25 28 30]
[ 0  2  7  9 16 19 21 22 24 25 28 30 31]


In [33]:
a =  np.random.rand(31, 1)
# print(a)
mutation_rate=0.5
mutation_mask = (np.random.rand(dim, 1) < mutation_rate).astype(float)
# print(mutation_mask)
# 统计 True 值的数量
num_true_values = np.count_nonzero(mutation_mask)

print(num_true_values)

15


In [3]:
import numpy as np 
alpha = np.random.uniform(0, 1, size=31)
# print(alpha)
mu, sigma = 0, 0.01
a = np.random.normal(mu, sigma, size=31)
print(a)

[ 0.00469152  0.00659777 -0.00164423  0.01350225 -0.01555822 -0.01264883
  0.00393513  0.00288629  0.00335959  0.00249217  0.00519367 -0.0021006
 -0.016811    0.02197975 -0.00618664 -0.00905247 -0.00557091  0.00704781
  0.00826932  0.00794601  0.01208622 -0.00980471  0.0060093  -0.00420214
  0.00525824 -0.01070709 -0.00173743  0.02113231  0.00587889  0.00470177
 -0.00282406]


In [7]:
import numpy as np 
from scipy.special import gamma as G

# 辅助-LEVY函数.    这个函数的目的是生成服从 Lévy 分布的随机数，通常用于优化算法中的多项式变异等操作。
def levy(beta, n):   # beta=0.3, n=len(p1)=31
    num = G(1 + beta) * np.sin(np.pi * beta / 2)
    den = G((1 + beta) / 2) * beta * 2 ** ((beta - 1) / 2)  # 公式6
    sigma_u, sigma_v = (num / den) ** (1 / beta), 1
    u = np.random.normal(0, sigma_u, size=n)
    v = np.random.normal(0, sigma_v, size=n)
    z = u / (np.abs(v) ** (1 / beta))
    return z.reshape(n, 1)

# 修正子代函数
def repair(y, lb, ub):  # 刚传进来的y.shape=(31, 1), max=1348.03   min=-0.007
    for i in range(len(y)):
        y[i] = max(y[i], lb[i])  #负数变为0, 
    s = np.sum(y)   
    if s > 1e-04:
        y = y / s    #缩放使所有变量总和=1,  归一化操作
    else:
        # print("Repair Error: all components are 0!")
        y = solution(lb, ub)  #随机生成y  根据上下界生成均匀分布的解(31,1)
    return y


def crossover(p1, p2):
    if np.random.uniform(0, 1) < 1:
        alpha = np.random.uniform(0, 1, size=p1.shape)
        return alpha * p1 + (1 - alpha) * p2
    else:
        return p1

def adj_lvxm_improved(P, i, b, lb, ub, par):
    gamma, delta, alpha, beta, pm, etam, epsilon, mu, sigma = par
    np.random.shuffle(b)
    p1 = P[i]
    p2 = P[b[0]]
    y = p1 * epsilon + alpha * levy(beta, len(p1)) * (p1 - p2) * (1 - epsilon)
    y = repair(y, lb, ub)
    y = crossover(y, p2)
    y = poly_mutation(y, lb, ub, etam, pm)
    # y = gaussian_mutation(y, mu, sigma)
    y = repair(y, lb, ub)
    return y


In [8]:
import numpy as np

# ... 上述函数保持不变 ...

def simulated_annealing_acceptance(new_fitness, current_fitness, temperature):
    if new_fitness < current_fitness:
        return True
    else:
        delta = new_fitness - current_fitness
        probability = np.exp(-delta / temperature)
        return np.random.uniform(0, 1) < probability

def genetic_algorithm_with_SA(objective_function, population_size, max_iterations, lb, ub, par):
    P = np.random.uniform(lb, ub, (population_size, len(lb)))  # Initialize population
    fitness = np.apply_along_axis(objective_function, 1, P)
    print(fitness.shape)
    best_index = np.argmin(fitness)
    best_solution = P[best_index]   # 最好的解
    best_fitness = fitness[best_index]  # 最好的目标函数值

    initial_temperature = 1.0
    cooling_rate = 0.995
    
    for iteration in range(max_iterations):
        temperature = initial_temperature * (cooling_rate ** iteration)

        for i in range(population_size):
            b = list(range(population_size))
            b.remove(i)
            y = adj_lvxm_improved(P, i, b, lb, ub, par)
            y_fitness = objective_function(y)

            # Simulated Annealing acceptance criterion
            if simulated_annealing_acceptance(y_fitness, fitness[i], temperature):
                P[i] = y
                fitness[i] = y_fitness

                # Update best solution if necessary
                if y_fitness < best_fitness:
                    best_fitness = y_fitness
                    best_solution = y

    return best_solution, best_fitness

# Example usage
def objective_function(x):
    return np.sum(x ** 2)

population_size = 50
max_iterations = 1000
lb = [-10, -10]
ub = [10, 10]
par = (0.5, 0.5, 0.5, 1.5, 0.1, 10, 0.5, 0.5, 0.5)  # Example parameters

best_solution, best_fitness = genetic_algorithm_with_SA(objective_function, population_size, max_iterations, lb, ub, par)
print("Best solution:", best_solution)
print("Best fitness:", best_fitness)


(50,)


ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()

In [3]:
import numpy as np

b = np.arange(10)
print(type(b))
print(b)
b = b[b != 2]
print(b)


a = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13]
print(type(a))

<class 'numpy.ndarray'>
[0 1 2 3 4 5 6 7 8 9]
[0 1 3 4 5 6 7 8 9]
<class 'list'>
